In [ ]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()


In [ ]:

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

api_version = os.environ["AZURE_OPENAI_API_VERSION"]
endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]

client = AzureOpenAI(
    api_version=api_version,
    azure_ad_token_provider=token_provider,
)
response = client.responses.create(
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],
    input="未来のテクノロジートレンドを3つ挙げてください"
)
print(response.output_text)

## Chaining responses together

In [ ]:
inputs = [{"type": "message", "role": "user", "content": "Define and explain the concept of catastrophic forgetting?"}] 
  
response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"], # replace with your model deployment name  
    input=inputs  
)  
  
inputs += response.output

inputs.append({"role": "user", "type": "message", "content": "Explain this at a level that could be understood by a college freshman"}) 
               

second_response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],  
    input=inputs
)  
      
print(second_response.model_dump_json(indent=2)) 

## Chaining responses manually

In [ ]:
inputs = [{"type": "message", "role": "user", "content": "Define and explain the concept of catastrophic forgetting?"}] 
  
response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],  # replace with your model deployment name  
    input=inputs  
)  
  
inputs += response.output

inputs.append({"role": "user", "type": "message", "content": "Explain this at a level that could be understood by a college freshman"}) 
               

second_response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],  
    input=inputs
)  
      
print(second_response.model_dump_json(indent=2))  

## Function calling

In [ ]:
response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],  # replace with your model deployment name  
    tools=[  
        {  
            "type": "function",  
            "name": "get_weather",  
            "description": "Get the weather for a location",  
            "parameters": {  
                "type": "object",  
                "properties": {  
                    "location": {"type": "string"},  
                },  
                "required": ["location"],  
            },  
        }  
    ],  
    input=[{"role": "user", "content": "What's the weather in San Francisco?"}],  
)  

print(response.model_dump_json(indent=2))  
  
# To provide output to tools, add a response for each tool call to an array passed  
# to the next response as `input`  
input = []  
for output in response.output:  
    if output.type == "function_call":  
        match output.name:  
            case "get_weather":  
                input.append(  
                    {  
                        "type": "function_call_output",  
                        "call_id": output.call_id,  
                        "output": '{"temperature": "70 degrees"}',  
                    }  
                )  
            case _:  
                raise ValueError(f"Unknown function call: {output.name}")  
  
second_response = client.responses.create(  
    model=os.environ["AZURE_OPENAI_MODEL_NAME"],  
    previous_response_id=response.id,  
    input=input  
)  

print(second_response.model_dump_json(indent=2)) 

## List input items

In [ ]:
list_parameter="resp_67d856fcfba0819081fd3cffee2aa1c0"
response = client.responses.input_items.list(list_parameter)

print(response.model_dump_json(indent=2))

NameError: name 'client' is not defined